In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dutils import get_current_date
import os
from azure.identity.aio import (ClientSecretCredential, DefaultAzureCredential,
                                get_bearer_token_provider)
from autogen_agentchat.agents import UserProxyAgent
from fmputils import *
from yfutils import *
from autogen_core.tools import FunctionTool
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from datetime import date, timedelta, datetime

In [2]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

True

In [3]:
def GetTokenProvider(scopes):
    return get_bearer_token_provider(DefaultAzureCredential(), scopes)

In [4]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    model="gpt-4o",
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    # azure_ad_token_provider=GetTokenProvider(
    #                 "https://cognitiveservices.azure.com/.default"
    #             ),
    api_key=os.getenv('AZURE_OPENAI_KEY'),
    model_capabilities={
        "vision": False,
        "function_calling": True,
        "json_output": True,
    }
)

In [5]:
get_company_info = FunctionTool(yfUtils.get_company_info, description="get a company's profile information")
get_stock_data = FunctionTool(yfUtils.get_stock_data, description="retrieve stock price data for designated ticker symbol")
get_financial_metrics = FunctionTool(fmpUtils.get_financial_metrics, description="get latest financial basics for a designated company")
get_company_news = FunctionTool(fmpUtils.get_company_news, description="retrieve market news related to designated company")
get_analyst_recommendations = FunctionTool(yfUtils.get_analyst_recommendations, description="get analyst recommendation for a designated company")

In [6]:
analyst_agent = AssistantAgent(
    name="Market_Analyst",
    system_message="As a Market Analyst, one must possess strong analytical and problem-solving abilities, collect necessary financial information and aggregate them based on client's requirement."
    "For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    model_client=model_client,
    tools=[get_company_info, get_stock_data, get_financial_metrics, get_analyst_recommendations, get_company_news]
)

In [7]:
url = f"https://financialmodelingprep.com/api/v4/batch_earning_call_transcript/AAPL?year=2024&apikey=4121cd4474b95b5c315eaf4cd3189606"

response = requests.get(url)
transcripts=[]

if response.status_code == 200:
    data = response.json()
    transcripts = [
        {
            "quarter": n["quarter"],
            "year": n["year"],
            "date": n["date"],
            "content": n["content"],
        }
        for n in data
    ]
    output = pd.DataFrame(transcripts)
    print(output)

   quarter  year                 date  \
0        4  2024  2024-10-31 17:00:00   
1        3  2024  2024-08-01 17:00:00   
2        2  2024  2024-05-02 17:00:00   
3        1  2024  2024-02-01 17:00:00   

                                             content  
0  Suhasini Chandramouli: Good afternoon, and wel...  
1  Suhasini Chandramouli: Good afternoon and welc...  
2  Suhasini Chandramouli: Good Afternoon, and wel...  
3  Operator: Good day, and welcome to the Apple Q...  


In [8]:
print(fmpUtils.get_earning_calls("AAPL"))

FMP api key found successfully.
   quarter  year                 date  \
0        4  2024  2024-10-31 17:00:00   
1        3  2024  2024-08-01 17:00:00   
2        2  2024  2024-05-02 17:00:00   
3        1  2024  2024-02-01 17:00:00   

                                             content  
0  Suhasini Chandramouli: Good afternoon, and wel...  
1  Suhasini Chandramouli: Good afternoon and welc...  
2  Suhasini Chandramouli: Good Afternoon, and wel...  
3  Operator: Good day, and welcome to the Apple Q...  


In [8]:
team = RoundRobinGroupChat([analyst_agent], max_turns=3)

In [9]:
# company = "Tesla"
company = "Microsoft"
stream = team.run_stream(task=f"Use all the tools provided to retrieve information available for {company} upon {get_current_date()}. Analyze the positive developments and potential concerns of {company} "
        "with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. "
        f"Then make a rough prediction (e.g. up/down by 2-3%) of the {company} stock price movement for next week. Provide a summary analysis to support your prediction.")
await Console(stream)

---------- user ----------
Use all the tools provided to retrieve information available for Microsoft upon 2025-01-18. Analyze the positive developments and potential concerns of Microsoft with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. Then make a rough prediction (e.g. up/down by 2-3%) of the Microsoft stock price movement for next week. Provide a summary analysis to support your prediction.
FMP api key found successfully.
FMP api key found successfully.
---------- Market_Analyst ----------
[FunctionCall(id='call_V3baVXCrm7aNoIeEy9VeKtsd', arguments='{"symbol": "MSFT"}', name='get_company_info'), FunctionCall(id='call_YDxYoPOj5sXeZ3x4hXXCBksz', arguments='{"symbol": "MSFT", "start_date": "2025-01-11", "end_date": "2025-01-18"}', name='get_stock_data'), FunctionCall(id='call_KI96nZCopZD1cAGziajCldbP', arguments='{"ticker_symbol": "MSFT"}', name='get_financial_metrics'), FunctionCall(id='call_TqLyE6earWKvGH4

d:\repos\secagent\src\backend\yfutils.py:34: RuntimeWarning: coroutine 'save_output' was never awaited
  save_output(stock_data, f"Stock data for {ticker.ticker}", save_path)


---------- Market_Analyst ----------
[FunctionExecutionResult(content='            Company Name                   Industry      Sector  \\\n0  Microsoft Corporation  Software - Infrastructure  Technology   \n\n         Country                    Website  \n0  United States  https://www.microsoft.com  ', call_id='call_V3baVXCrm7aNoIeEy9VeKtsd'), FunctionExecutionResult(content='                                 Open        High         Low       Close  \\\nDate                                                                        \n2025-01-13 00:00:00-05:00  415.239990  418.500000  412.290009  417.190002   \n2025-01-14 00:00:00-05:00  417.809998  419.739990  410.720001  415.670013   \n2025-01-15 00:00:00-05:00  419.130005  428.149994  418.269989  426.309998   \n2025-01-16 00:00:00-05:00  428.700012  429.489990  424.390015  424.579987   \n2025-01-17 00:00:00-05:00  434.089996  434.480011  428.170013  429.029999   \n\n                             Volume  Dividends  Stock Splits  \nDate   

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Use all the tools provided to retrieve information available for Microsoft upon 2025-01-18. Analyze the positive developments and potential concerns of Microsoft with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. Then make a rough prediction (e.g. up/down by 2-3%) of the Microsoft stock price movement for next week. Provide a summary analysis to support your prediction.', type='TextMessage'), ToolCallRequestEvent(source='Market_Analyst', models_usage=RequestUsage(prompt_tokens=519, completion_tokens=149), content=[FunctionCall(id='call_V3baVXCrm7aNoIeEy9VeKtsd', arguments='{"symbol": "MSFT"}', name='get_company_info'), FunctionCall(id='call_YDxYoPOj5sXeZ3x4hXXCBksz', arguments='{"symbol": "MSFT", "start_date": "2025-01-11", "end_date": "2025-01-18"}', name='get_stock_data'), FunctionCall(id='call_KI96nZCopZD1cAGziajCldbP', arguments='{"